In [18]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision.io import read_image
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, ConcatDataset

# Device configuration
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

has_gpu = torch.cuda.is_available()
has_mps = getattr(torch,'has_mps',False)
device = "mps" if getattr(torch,'has_mps',False) \
    else "cuda" if torch.cuda.is_available() else "cpu"

print(device)


# Hyper-parameters 
epochs = 2000
image_size = (360, 640)
batch_size = 16
learning_rate = 0.000001
momentum = 0.9

mps


In [7]:
#Data preparation
class CustomImageDataset(Dataset):
    def __init__(self, labels_arr, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.DataFrame(labels_arr)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = torch.tensor(int(self.img_labels.iloc[idx, 1]))
        if self.transform:
            image = self.transform(image.float())
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
    
    
def create_dataset(folder, label):
    files = os.listdir(folder)
    for i in files:
        if i =='.DS_Store':
            files.remove(i)
    files = np.array(files)
    labels_arr = np.zeros(len(files), dtype=np.uint8)
    labels_arr.fill(label)
    labels_arr = np.column_stack((files, labels_arr))

    transform = transforms.Compose([transforms.Resize(image_size)])
    dataset = CustomImageDataset(labels_arr, folder, transform=transform)
    return dataset

In [8]:
#pure train datasets
dataset_low = create_dataset("/Users/gaky/Desktop/efir/on for use/true low", int(0))
dataset_off = create_dataset("/Users/gaky/Desktop/efir/off for use/off mix", int(1))


#validation datasets
dataset_off_val = create_dataset("/Users/gaky/Desktop/efir/off for use/validation off mix", int(1))
dataset_low_val = create_dataset("/Users/gaky/Desktop/efir/on for use/low valid", int(0))

train_dataset = ConcatDataset([dataset_low, dataset_off])
print(len(train_dataset))

dataloader_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

dataloader_val_off = DataLoader(dataset_off_val, batch_size=len(dataset_off_val), shuffle=False)
dataloader_low_val = DataLoader(dataset_low_val, batch_size=len(dataset_low_val), shuffle=False)

dataloader_off = DataLoader(dataset_off, batch_size=len(dataset_off), shuffle=False)
dataloader_low = DataLoader(dataset_low, batch_size=len(dataset_low), shuffle=False)

302


In [9]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
                        nn.BatchNorm2d(out_channels))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [10]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 10):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv2d(3, 64, kernel_size = 7, stride = 2, padding = 3),
                        nn.BatchNorm2d(64),
                        nn.ReLU())
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512*6*14, 3)
        
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        #print(x.shape)
        x = x.view(-1, 512*6*14)
        x = self.fc(x)

        return x

In [22]:
model = ResNet(ResidualBlock, [3, 4, 6, 3]).to(device)



# Loss and optimizer
weights = torch.tensor([3.0, 1.0])
loss_function = nn.CrossEntropyLoss(weight=weights)

optim = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.001, momentum = 0.9)
#optim = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1)

In [12]:
def save_model(epoch):
    model_scripted = torch.jit.script(model) # Export to TorchScript
    model_scripted.save("/Users/gaky/Desktop/efir/Models/strange model/"+ str(epoch) + ".pth")

In [13]:
def save_model2(epoch):
    torch.save(model.state_dict(), "/Users/gaky/Desktop/efir/Models/binary resnet weighted/"+ str(epoch) + ".pth")

In [14]:
def evaluate(dataloader, name):
    with torch.no_grad():
        dataset = iter(dataloader)
        x, y = next(dataset)
        
        x = x/255.
        x = x.to(device)
        y = y.to(device)
        model.eval()
        out = model(x.float())
        cat = torch.argmax(out, dim=1)
        
        if name == "off_val":
            for i in range(len(cat)):
                if cat[i] == 2:
                    cat[i] = 1

        accuracy = (cat == y).float().mean()
        print(accuracy, name)

In [15]:
def evaluate2(dataloader, name):
    with torch.no_grad():
        dick = []
        for i, (x, y) in enumerate(dataloader):
        
            x = x/255.
            x = x.to(device)
            y = y.to(device)
            model.eval()
            out = model(x.float())
            cat = torch.argmax(out, dim=1)


            if name == "off":
                for i in range(len(cat)):
                    if cat[i] == 2:
                        cat[i] = 1
                        
            accuracy = (cat == y).float()
            dick.append(accuracy.mean())
        print(np.mean(dick), name)

In [19]:
losses = []
import time

for epoch in range(epochs):
    start = time.time()
    for i, (x, y) in enumerate(dataloader_train):
        x = x/255.
        x = x.to(device)
        y = y.to(device)
        model.train()
        
        model.zero_grad()
        start = time.time()
        
        out = model(x.float())
        
        loss = loss_function(out, y.long())
        loss_mean = loss.mean()
        loss_mean.backward()
        optim.step()

        print(epoch, i, loss_mean)
        
        losses.append(loss.detach().item())
        
        if epoch % 1 == 0 and epoch != 0 and i == 0:
            evaluate(dataloader_val_off, "off_val")
            evaluate(dataloader_low_val, "low_val")
            
            #evaluate2(dataloader_off, "off")
            #evaluate2(dataloader_low, "low")
            
            save_model2(epoch)
            #evaluate(dataloader_off, "off")
            #evaluate(dataloader_low, "low")
            #evaluate(dataloader_on, "on")
            #evaluate(dataloader_night, "night")
            
    stop = time.time()
    print("time: ", stop - start)
    print("loss: ", np.mean(losses))

0 0 tensor(0.0095, device='mps:0', grad_fn=<MeanBackward0>)
0 1 tensor(0.0073, device='mps:0', grad_fn=<MeanBackward0>)
0 2 tensor(0.0104, device='mps:0', grad_fn=<MeanBackward0>)
0 3 tensor(0.0027, device='mps:0', grad_fn=<MeanBackward0>)
0 4 tensor(0.0296, device='mps:0', grad_fn=<MeanBackward0>)
0 5 tensor(0.0050, device='mps:0', grad_fn=<MeanBackward0>)
0 6 tensor(0.0018, device='mps:0', grad_fn=<MeanBackward0>)
0 7 tensor(0.0011, device='mps:0', grad_fn=<MeanBackward0>)
0 8 tensor(0.0449, device='mps:0', grad_fn=<MeanBackward0>)
0 9 tensor(0.0215, device='mps:0', grad_fn=<MeanBackward0>)
0 10 tensor(0.0481, device='mps:0', grad_fn=<MeanBackward0>)
0 11 tensor(0.0061, device='mps:0', grad_fn=<MeanBackward0>)
0 12 tensor(0.0002, device='mps:0', grad_fn=<MeanBackward0>)
0 13 tensor(0.0266, device='mps:0', grad_fn=<MeanBackward0>)
0 14 tensor(0.1070, device='mps:0', grad_fn=<MeanBackward0>)
0 15 tensor(0.0127, device='mps:0', grad_fn=<MeanBackward0>)
0 16 tensor(0.0010, device='mps:0'

6 9 tensor(0.0046, device='mps:0', grad_fn=<MeanBackward0>)
6 10 tensor(0.0027, device='mps:0', grad_fn=<MeanBackward0>)
6 11 tensor(0.0214, device='mps:0', grad_fn=<MeanBackward0>)
6 12 tensor(0.0015, device='mps:0', grad_fn=<MeanBackward0>)
6 13 tensor(0.0018, device='mps:0', grad_fn=<MeanBackward0>)
6 14 tensor(0.0010, device='mps:0', grad_fn=<MeanBackward0>)
6 15 tensor(0.0110, device='mps:0', grad_fn=<MeanBackward0>)
6 16 tensor(0.0028, device='mps:0', grad_fn=<MeanBackward0>)
6 17 tensor(0.0033, device='mps:0', grad_fn=<MeanBackward0>)
6 18 tensor(0.0011, device='mps:0', grad_fn=<MeanBackward0>)
time:  3.7507777214050293
loss:  0.014073611227042721
7 0 tensor(0.0313, device='mps:0', grad_fn=<MeanBackward0>)
tensor(0.9344, device='mps:0') off_val
tensor(0.9286, device='mps:0') low_val
7 1 tensor(0.0004, device='mps:0', grad_fn=<MeanBackward0>)
7 2 tensor(0.0025, device='mps:0', grad_fn=<MeanBackward0>)
7 3 tensor(0.0055, device='mps:0', grad_fn=<MeanBackward0>)
7 4 tensor(0.0046, 

KeyboardInterrupt: 

In [ ]:
losses = []
import time

for epoch in range(epochs):
    start = time.time()
    for i, (x, y) in enumerate(dataloader_train):
        x = x/255.
        x = x.to(device)
        y = y.to(device)
        model.train()
        
        model.zero_grad()
        start = time.time()
        
        out = model(x.float())
        
        loss = loss_function(out, y.long())
        loss_mean = loss.mean()
        loss_mean.backward()
        optim.step()

        print(epoch, i, loss_mean)
        
        losses.append(loss.detach().item())
        
        if epoch % 1 == 0 and epoch != 0 and i == 0:
            evaluate(dataloader_val_off, "off_val")
            evaluate(dataloader_low_val, "low_val")
            
            #evaluate2(dataloader_off, "off")
            #evaluate2(dataloader_low, "low")
            
            save_model2(epoch)
            #evaluate(dataloader_off, "off")
            #evaluate(dataloader_low, "low")
            #evaluate(dataloader_on, "on")
            #evaluate(dataloader_night, "night")
            
    stop = time.time()
    print("time: ", stop - start)
    print("loss: ", np.mean(losses))